In [2]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML

In [3]:
!ls ../../ppc/scripts

data				  hyperparameters
eval_models_aws.sh		  launch_autotunes.sh
eval_models_local_pierre.sh	  model_eval_zi_12000.sh
eval_models.sh			  nb_dataset
eval_synthetic_models_no_tune.sh  ppc_all.sh
eval_synthetic_models.sh	  zinb_dataset


In [4]:
data_path = '/home/pierre/scVI/data/PPC/results/'
dataset_name = 'log_poisson_nb_dataset_12000'
data_files = sorted(glob.glob(os.path.join(data_path, '{}/*.csv'.format(dataset_name))))
for data_file in data_files:
    print(data_file)
data_names = ['nb', 'zinb']

/home/pierre/scVI/data/PPC/results/log_poisson_nb_dataset_12000/nb_log_poisson_nb_dataset_12000.csv
/home/pierre/scVI/data/PPC/results/log_poisson_nb_dataset_12000/zinb_log_poisson_nb_dataset_12000.csv


In [5]:
dfs = []
for data_name, f in zip(data_names, data_files):
    if data_name != 'zifa_full':
        
        my_df = pd.read_csv(f, sep='\t')
#         my_df = pd.read_pickle(f)
        my_df.loc[:, 'model'] = data_name
        dfs.append(my_df)
        print(data_name, f)
#         display(my_df)
df = pd.concat(dfs, axis=0)

nb /home/pierre/scVI/data/PPC/results/log_poisson_nb_dataset_12000/nb_log_poisson_nb_dataset_12000.csv
zinb /home/pierre/scVI/data/PPC/results/log_poisson_nb_dataset_12000/zinb_log_poisson_nb_dataset_12000.csv


/home/pierre/miniconda3/envs/scvi/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [6]:
# df

In [7]:
def str_to_array(string):
    list_nbs = string[1:-1].replace('\n','').replace('  ',' ').split(' ')
    ind = 0
    
    while ind < len(list_nbs):
        if list_nbs[ind] == '':
            del list_nbs[ind]
        else:
            ind += 1
    
    print(len(list_nbs))
            
    for ind in range(len(list_nbs)):
        list_nbs[ind] = float(list_nbs[ind])
    return np.array(list_nbs)

gene_dropout_all = df[df.model == 'zinb'].zero_params_zero_scales_all
gene_dropout_all_cont = np.vstack(list(gene_dropout_all))
print(gene_dropout_all_cont, gene_dropout_all_cont.shape)
print(dataset_name + "_2 : ",  gene_dropout_all_cont.mean())

In [8]:
metrics = ['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']
h1_hypothesis = ['larger', 'larger', 'larger', 'larger', 'larger']
h1_hypothesis_bis = ['smaller', 'smaller', 'smaller', 'smaller', 'smaller']

df_nb = df.loc[df.model=='nb', metrics]
df_zinb = df.loc[df.model=='zinb', metrics]



from statsmodels.stats.weightstats import ttest_ind

def get_pvals(gby, other_df):
    my_df = gby[metrics]
    print(gby)
    assert my_df.shape[1] == len(metrics)
    pvals = []
    for idx, alternative in enumerate(h1_hypothesis):
        assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
        _, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
        pvals.append(pval)
    return np.array(pvals)

pvals_against_zinb = df.groupby('model').apply(get_pvals, other_df=df_zinb)
pvals_against_zinb = (pvals_against_zinb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

pvals_against_nb = df.groupby('model').apply(get_pvals, other_df=df_nb)
pvals_against_nb = (pvals_against_nb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    Unnamed: 0 gene_dropout_gene_dropout_logits_all  \
0            0                                  NaN   
1            1                                  NaN   
2            2                                  NaN   
3            3                                  NaN   
4            4                                  NaN   
5            5                                  NaN   
6            6                                  NaN   
7            7                                  NaN   
8            8                                  NaN   
9            9                                  NaN   
10          10                                  NaN   
11          11                                  NaN   
12          12                                  NaN   
13          13                                  NaN   
14          14                                  NaN   
15          15                                  NaN   
16          16                                  NaN   
17        

In [9]:
a = df.loc[df.model=='zifa_full', 'imputation_median_imputation_score']
b = df.loc[df.model=='zinb', 'imputation_median_imputation_score']
print(a.mean(), b.mean())
ttest_ind(a, b, alternative='smaller')

nan 4.377269649505616


/home/pierre/miniconda3/envs/scvi/lib/python3.6/site-packages/statsmodels/stats/weightstats.py:132: RuntimeWarning: invalid value encountered in double_scalars
  return self.sum / self.sum_weights
/home/pierre/miniconda3/envs/scvi/lib/python3.6/site-packages/statsmodels/stats/weightstats.py:837: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(var_pooled * (1. / d1.nobs + 1. /d2.nobs))


(nan, nan, 48.0)

In [10]:
pvals_against_zinb

model,nb,zinb
ll_ll,2.088875e-02,0.5
imputation_median_imputation_score,7.957823e-01,0.5
t_dropout_ks_stat,2.476293e-10,0.5
t_ratio_ks_stat,3.169690e-02,0.5
t_cv_ks_stat,1.373082e-03,0.5


In [11]:
def get_summary(gby):
    res = {}
    res['mean'] = gby.mean()
    res['std'] = gby.std()
    res['pvals_against_nb'] = pvals_against_nb[gby.name]
    res['pvals_against_zi'] = pvals_against_zinb[gby.name]
    return pd.DataFrame(res).T

# df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
#                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
df_summary = df.groupby('model')[metrics].apply(get_summary)
df_summary


def my_styler(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
#     print(val)
    style = ['background-color: yellow' if v < 0.05 else '' for v in val]
    return style

df_summary = df_summary.stack().unstack(1).sort_index(level=1).swaplevel()
style = df_summary.style.apply(my_styler, subset=['pvals_against_nb', 'pvals_against_zi'], axis=1)
print(dataset_name)
style
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()
# df_summary.style.apply(my_styler, subset=pd.IndexSlice[:, 'pvals'], axis=0)
# df_summary = df_summary.style.apply(my_styler, subset=pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :], axis=1).render()
# df_summary.pivot(index=['model', ['mean', 'std', 'pvals']], columns=metrics)
# style


log_poisson_nb_dataset_12000


In [12]:
df_summary.reset_index(level=-1)

,model,mean,std,pvals_against_nb,pvals_against_zi
imputation_median_imputation_score,nb,4.431129,0.375012,0.500000,7.957823e-01
imputation_median_imputation_score,zinb,4.377270,0.264189,0.204218,5.000000e-01
ll_ll,nb,170.335087,4.978498,0.500000,2.088875e-02
ll_ll,zinb,171.841911,1.376412,0.979111,5.000000e-01
t_cv_ks_stat,nb,0.612400,0.266738,0.500000,1.373082e-03
t_cv_ks_stat,zinb,0.747600,0.160147,0.998627,5.000000e-01
t_dropout_ks_stat,nb,0.241600,0.108183,0.500000,2.476293e-10
t_dropout_ks_stat,zinb,0.366400,0.067906,1.000000,5.000000e-01
t_ratio_ks_stat,nb,0.242000,0.116286,0.500000,3.169690e-02
t_ratio_ks_stat,zinb,0.280000,0.083397,0.968303,5.000000e-01


# Export old Table

In [13]:
# # save as png
# import imgkit
# imgkit.from_string(style, os.path.join(data_path, '{}.png'.format(dataset_name)))

# text_file = open(os.path.join(data_path, 'latex_table.txt'), "w")
# text_file.write(df_summary.to_latex())
# text_file.close()

# New export Latex

In [22]:
df_summary.T

imputation_median_imputation_score                 ll_ll  \
model                                            nb      zinb          nb   
mean                                       4.431129  4.377270  170.335087   
std                                        0.375012  0.264189    4.978498   
pvals_against_nb                           0.500000  0.204218    0.500000   
pvals_against_zi                           0.795782  0.500000    0.020889   

                             t_cv_ks_stat           t_dropout_ks_stat  \
model                   zinb           nb      zinb                nb   
mean              171.841911     0.612400  0.747600      2.416000e-01   
std                 1.376412     0.266738  0.160147      1.081828e-01   
pvals_against_nb    0.979111     0.500000  0.998627      5.000000e-01   
pvals_against_zi    0.500000     0.001373  0.500000      2.476293e-10   

                           t_ratio_ks_stat            
model                 zinb              nb      zinb  
mean              0.366400        0.242000  0.280000  
std               0.067906        0.116286  0.083397  
pvals_against_nb  1.000000        0.500000  0.968303  
pvals_against_zi  0.500000        0.031697  0.500000

In [87]:
def boldizer(row):
    to_bold = (row.pvals_against_nb <= 5e-3) | (row.pvals_against_zi <= 5e-3)
#     print(to_bold)
    if to_bold:
        row.latex = '\mathbf{' + row.latex + '}'
    return row

In [93]:
n_dec = 3


work_df = (df_summary
           .reset_index(level=-1)
           .reset_index()
           .assign(latex=lambda x: x['mean'].round(n_dec).astype(str) + ' \pm ' + x['std'].round(n_dec).astype(str))
           .apply(boldizer, axis=1)
           .assign(Metric=lambda x: 
                   x.loc[:, 'index'].replace({
                       'imputation_median_imputation_score': 'Inputation Score',
                       'll_ll': 'Log Likelihood',
                       't_cv_ks_stat': 'PPC (Coefficient of Variation)',
                       't_dropout_ks_stat': 'PPC (Dropout)',
                       't_ratio_ks_stat': 'PPC (Ratio of zeros)'
           }),
                   latex=lambda x: '$' + x.latex + '$'
                  )
          )

work_df

,index,model,mean,std,pvals_against_nb,pvals_against_zi,latex,Metric
0,imputation_median_imputation_score,nb,4.431129,0.375012,0.500000,7.957823e-01,$4.431 \pm 0.375$,Inputation Score
1,imputation_median_imputation_score,zinb,4.377270,0.264189,0.204218,5.000000e-01,$4.377 \pm 0.264$,Inputation Score
2,ll_ll,nb,170.335087,4.978498,0.500000,2.088875e-02,$170.335 \pm 4.978$,Log Likelihood
3,ll_ll,zinb,171.841911,1.376412,0.979111,5.000000e-01,$171.842 \pm 1.376$,Log Likelihood
4,t_cv_ks_stat,nb,0.612400,0.266738,0.500000,1.373082e-03,$\mathbf{0.612 \pm 0.267}$,PPC (Coefficient of Variation)
5,t_cv_ks_stat,zinb,0.747600,0.160147,0.998627,5.000000e-01,$0.748 \pm 0.16$,PPC (Coefficient of Variation)
6,t_dropout_ks_stat,nb,0.241600,0.108183,0.500000,2.476293e-10,$\mathbf{0.242 \pm 0.108}$,PPC (Dropout)
7,t_dropout_ks_stat,zinb,0.366400,0.067906,1.000000,5.000000e-01,$0.366 \pm 0.068$,PPC (Dropout)
8,t_ratio_ks_stat,nb,0.242000,0.116286,0.500000,3.169690e-02,$0.242 \pm 0.116$,PPC (Ratio of zeros)
9,t_ratio_ks_stat,zinb,0.280000,0.083397,0.968303,5.000000e-01,$0.28 \pm 0.083$,PPC (Ratio of zeros)


In [94]:
pivot = pd.pivot(work_df, index='Metric', values='latex', columns='model')
pivot

model,nb,zinb
Metric,,
Inputation Score,$4.431 \pm 0.375$,$4.377 \pm 0.264$
Log Likelihood,$170.335 \pm 4.978$,$171.842 \pm 1.376$
PPC (Coefficient of Variation),$\mathbf{0.612 \pm 0.267}$,$0.748 \pm 0.16$
PPC (Dropout),$\mathbf{0.242 \pm 0.108}$,$0.366 \pm 0.068$
PPC (Ratio of zeros),$0.242 \pm 0.116$,$0.28 \pm 0.083$


In [91]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, Inputation Score to PPC (Ratio of zeros)
Data columns (total 2 columns):
nb      5 non-null object
zinb    5 non-null object
dtypes: object(2)
memory usage: 120.0+ bytes


In [99]:
res = pivot.to_latex(escape=False)
res

'\\begin{tabular}{lll}\n\\toprule\nmodel &                          nb &                 zinb \\\\\nMetric                         &                             &                      \\\\\n\\midrule\nInputation Score               &           $4.431 \\pm 0.375$ &    $4.377 \\pm 0.264$ \\\\\nLog Likelihood                 &         $170.335 \\pm 4.978$ &  $171.842 \\pm 1.376$ \\\\\nPPC (Coefficient of Variation) &  $\\mathbf{0.612 \\pm 0.267}$ &     $0.748 \\pm 0.16$ \\\\\nPPC (Dropout)                  &  $\\mathbf{0.242 \\pm 0.108}$ &    $0.366 \\pm 0.068$ \\\\\nPPC (Ratio of zeros)           &           $0.242 \\pm 0.116$ &     $0.28 \\pm 0.083$ \\\\\n\\bottomrule\n\\end{tabular}\n'

In [98]:
# import io #Used as buffer


# def convertToLaTeX(df, alignment="c"):
#     """
#     Convert a pandas dataframe to a LaTeX tabular.
#     Prints labels in bold, does not use math mode
#     """
#     numColumns = df.shape[1]
#     numRows = df.shape[0]
#     output = io.StringIO()
#     colFormat = ("%s|%s" % (alignment, alignment * numColumns))
#     #Write header
#     output.write("\\begin{tabular}{%s}\n" % colFormat)
#     columnLabels = ["\\textbf{%s}" % label for label in df.columns]
#     output.write("& %s\\\\\\hline\n" % " & ".join(columnLabels))
#     #Write data lines
#     for i in range(numRows):
#         output.write("\\textbf{%s} & %s\\\\\n"
#                      % (df.index[i], " & ".join([str(val) for val in df.ix[i]])))
#     #Write footer
#     output.write("\\end{tabular}")
#     return output.getvalue()

# convertToLaTeX(pivot)

In [100]:
path = os.path.join(data_path, '{}.tex'.format(dataset_name))
with open(path, 'w') as tf:
     tf.write(res)

In [75]:
path

'/home/pierre/scVI/data/PPC/results/log_poisson_nb_dataset_12000.tex'